<a href="https://colab.research.google.com/github/eric19950925/PythonColabTTU/blob/main/PythonTTU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

練習1:直接爬原始碼 HTML response


In [ ]:
from urllib import request
url = "https://ctee.com.tw/livenews/aj"
src_url = "https://github.com/eric19950925/PythonColabTTU/blob/main/PythonTTU.ipynb"
res = request.urlopen(url)
content = res.read().decode('utf-8')
print(content)

練習2:爬指定字串 Scraping and Parsing the data with parser

In [10]:
from bs4 import BeautifulSoup
import requests as rq
r = rq.get(url)
soup = BeautifulSoup(r.text,"lxml")
print(r)
print(soup.find('h1').text)

<Response [200]>



工商時報 - 閱讀財經新聞的第一選擇




例外處理 Exception Handling

In [11]:
def get_tag_text(url,tag):
  try:
    resp = rq.get(url)
    if resp.status_code == 200:
      soup = BeautifulSoup(r.text,"lxml")
      return soup.find(tag).text
  except Exception as e:
    print('Exception:%s'%(e))
  return None

bad_url = ""
print(get_tag_text(url,'h1'))
print(get_tag_text(url,'h88'))
print(get_tag_text(bad_url,'h1'))




工商時報 - 閱讀財經新聞的第一選擇


None
Exception:Invalid URL '': No schema supplied. Perhaps you meant http://?
None


練習3:新聞文章彙整


In [13]:
def get_web_page(url):
  resp = rq.get(
      url = url,
      cookies = {'over18': '1'}
  )
  if resp.status_code != 200:
    print('invalid url:',resp.url)
    return None
  else: 
    return resp.text

In [17]:
def get_articles(dom,date):
  soup = BeautifulSoup(dom,'html5lib')

  paging_div = soup.find('div','btn-group btn-group-paging')
  prev_url = paging_div.find_all('a')[1]['href']

  articles = []
  divs = soup.find_all('div','r-ent')

  for d in divs:
    if d.find('div','date').text.strip() == date:
      push_count = 0
      push_str = d.find('div','nrec').text

      if push_str:
        try:
          push_count = int(push_str)
        except ValueError:
          if push_str == '爆':
              push_count == 99
          elif push_str.startswith('X'):
            push_count == -10

      if d.find('a'):
        href = d.find('a')['href']
        title = d.find('a').text
        author = d.find('div','author').text if d.find('div','author') else ''
        articles.append({
            'title':title,
            'href':href,
            'push_count':push_count,
            'author':author
            })

  return articles,prev_url

In [ ]:
PTT_URL = 'https://www.ptt.cc'
import time
import json

current_page=get_web_page(PTT_URL + '/bbs/Gossiping/index.html')

if current_page:
  articles = []

  today = time.strftime("%m/%d").lstrip('0')

  current_articles,prev_url = get_articles(current_page,today)

  while current_articles:
    articles += current_articles
    current_page = get_web_page(PTT_URL + prev_url)
    current_articles,prev_url = get_articles(current_page,today)

  print('今天有',len(articles),'篇文章')
  threshold = 50
  print('熱門文章(>%d推):'%(threshold))
  for a in articles:
    if int(a['push_count'])>threshold:
      print(a)
  
  with open('gossiping.json','w',encoding='utf-8') as f:
    json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

In [16]:
!pip install pyautogui

     |████████████████████████████████| 59 kB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... canceled
ERROR: Operation cancelled by user


In [ ]:
import pyautogui
import time
 
def click(): 
    time.sleep(0.1)     
    pyautogui.click()
    with mss(display=':0') as sct: 
      sct.shot()
 
def main():
    for i in range(20): 
        click()
 
main()

KeyError: ignored